# Assignment 2 Deepankar Varma 102003431 

In [1]:
import cv2
import numpy as np
import os

# Load dataset
dataset_path = 'Images'
data = []
labels = []
for folder_name in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder_name)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        data.append(gray)
        labels.append(folder_name)

# Preprocess data
data = np.array(data)
labels = np.array(labels)
data = data.reshape((data.shape[0], -1))
labels = labels.reshape((labels.shape[0], -1))

# Train model
model = cv2.face.LBPHFaceRecognizer_create()
model.train(data, labels)

# Test model
test_img = cv2.imread('test1.jpg')
gray_test_img = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
label, confidence = model.predict(gray_test_img)
print(label, confidence)